# Leif Kjos ***REMOVED*** BI Analyst Take Home Test
## Setup
This notebook contains the code to create a database in postgres and import flight data. In produciton, I normally would not use a Juypter notebook, but it makes it easy to present my code in this demo.

The first cell below contains all our necessary imports and loads our environment variables. Be sure to run the cells in order or press the run all button up top, otherwise you may get errors.

In [ ]:
import chardet, csv, datetime, os
from copy import deepcopy
from DBConn import DBConn
from dotenv import load_dotenv
from helpers import make_table, row_count, print_max_str_len
import numpy as np
import pandas as pd
from sqlalchemy.dialects.postgresql import DOUBLE_PRECISION, SMALLINT, BOOLEAN, VARCHAR, CHAR
from sqlalchemy import create_engine, event


PROJECT_ROOT = os.path.dirname(os.path.realpath("__file__")) #Change if needed, defaults to location of this file
print('Project Root:', PROJECT_ROOT)

## Name of database to create.
DATABASE_NAME = 'flight_data'
DBConn.set_database(DATABASE_NAME) 

## Set Data Directory
# DATA_DIR = os.path.join(PROJECT_ROOT,'data') # modify as needed or move data to projec_root\data\ (which is in .gitignore)
DATA_DIR = 'C:\\Users\\Public\\data\\' # modify as needed or move data to projec_root\data\ (which is in .gitignore)
print('Data Directory:', DATA_DIR)

## Set data paths -- modify folder/file names if yours are different
FLIGHT_DATA_DIR = os.path.join(DATA_DIR, 'FlightDataUncompressed') #assumes flights are stored in sub-directory called 'FlightDataUncompressed'
FLIGHT_DATA_FNAMES = os.listdir(FLIGHT_DATA_DIR)
FLIGHT_DATA_PATHS = []
for file_name in FLIGHT_DATA_FNAMES:
    FLIGHT_DATA_PATHS.append(os.path.join(FLIGHT_DATA_DIR, file_name))

# assumes airports and carriers are stored in data dir
AIRPORTS_PATH = os.path.join(DATA_DIR, 'airports.csv')
CARRIERS_PATH = os.path.join(DATA_DIR, 'carriers.csv')

# load enviornment variables storing database conneciton info, accessed via os.getenv('KEY_NAME')
load_dotenv(PROJECT_ROOT)   

# initialize answers for problem set
ans = dict()
head = dict() 

## EDA with Pandas -- determine structure of data for database schema


In [41]:
## First, lets see how big the files are and encoding
total_rows = 0
file_rows = dict()
for path_ in FLIGHT_DATA_PATHS:
    name = os.path.basename(path_)
    with open(path_,'rb') as file:
        if int(name.rstrip('.csv'))>=0:       
            file_rows[name] = row_count(file)
            total_rows += file_rows[name]
            print(f"{name}: {file_rows[name]}")
            # print(f'{name} chardet:') #to find encoding
            # print(chardet.detect(file.read(1000)))
print(f'total rows: {total_rows}')

1987.csv: 1311826
1988.csv: 5202096
1989.csv: 5041200
1990.csv: 5270893
1991.csv: 5076925
1992.csv: 5092157
1993.csv: 5070501
1994.csv: 5180048
1995.csv: 5327435
1996.csv: 5351983
1997.csv: 5411843
1998.csv: 5384721
1999.csv: 5527884
2000.csv: 5683047
2001.csv: 5967780
2002.csv: 5271359
2003.csv: 6488540
2004.csv: 7129270
2005.csv: 7140596
2006.csv: 7141922
2007.csv: 7453215
2008.csv: 7009728
total rows: 123534969


In [40]:
# ingest to pandas
airports_df = pd.read_csv(AIRPORTS_PATH) 
carriers_df = pd.read_csv(CARRIERS_PATH)
carriers_df = carriers_df.fillna('NA') ## Added after EDA, due to North American Airlines being interpreted as null

print(f'Found {len(FLIGHT_DATA_FNAMES)} years of flight data starting: from {FLIGHT_DATA_FNAMES[0].rstrip(".csv")} - {FLIGHT_DATA_FNAMES[-1].rstrip(".csv")}')
print(f'looking at {FLIGHT_DATA_FNAMES[0].rstrip(".csv")}')
flight_df = pd.read_csv(FLIGHT_DATA_PATHS[9], encoding='ascii')

Found 22 years of flight data starting: from 1987 - 2008
looking at 1987


In [42]:
## Figuring out airport schema
# uncomment one line at a time to see info about the dataframe
# print(airports_df.head(5))    # first five rows
# print(airports_df.info())     # column information and data types

## During import, I found that CMB was missing so I looked up the airport and added it manually in pgadmin
# print(airports_df[airports_df['iata'].astype(str).str.contains('CBM')])

# print_max_str_len(airports_df)  # max col length
print(airports_df.isnull().sum()) #count null per column


iata        0
airport     0
city       12
state      12
country     0
lat         0
long        0
dtype: int64


In [39]:
## Figuring out Carriers Schema
## uncomment lines to see what steps I took.

# carriers_df.head()                #first five rows
# carriers_df.info()                #info about data types
print_max_str_len(carriers_df)    #max string length

## Null count
# print('null values:')
# print(carriers_df.isnull().sum())

### looks like we have a null value
# carriers_df[carriers_df.Code.isnull()]

###and it's caused by North American being interpreted as NA.
## added null handling above on import step


0 Code str 7
1 Description str 83


In [ ]:
## Figuring out Flights Schema

# flight_df.info()
# flight_df.UniqueCarrier.isnull().sum()
# print_max_str_len(flight_df)

## This looks like a boolean field
# print(flight_df.Diverted.value_counts())

## fk to carriers
# flight_df.UniqueCarrier[0]

In [38]:
## More on flights
flight_df.head() 



,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1996,1,29,1,2039.0,1930,2245.0,2139,DL,345,...,6,10,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,1996,1,30,2,1931.0,1930,2142.0,2139,DL,345,...,5,22,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,1996,1,31,3,1956.0,1930,2231.0,2139,DL,345,...,7,27,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,1996,1,1,1,1730.0,1550,1909.0,1745,DL,411,...,4,14,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,1996,1,2,2,1714.0,1550,1841.0,1745,DL,411,...,4,8,0,NaN,0,NaN,NaN,NaN,NaN,NaN


Ok, so we've now looked at the columns in each dataset and can see that we'll need three tables relaations between:
- `flight.Origin`, `flight.Destination` to `airports.iata`
- `flight.UniqueCarrier` to `carrier.Code`

Now lets get the tables created in SQL:


### Create Flight Data Database
The database in the .env file is just an entry point. We are going to now create new database and load in our flight data.

#### Create database connection

In [ ]:
### Create Database


# Create database if it doesn't exist already
sql = f'''--sql
SELECT 'CREATE DATABASE {DATABASE_NAME}'
WHERE NOT EXISTS (SELECT FROM pg_database WHERE datname = '{DATABASE_NAME}');
'''

conn = DBConn()
result = conn.exec(sql)
print("Database created successfully or already exists....")
DBConn.set_database(DATABASE_NAME)
conn.close()


## Create Tables

In [ ]:
## Create Airport Table
conn = DBConn()
print('accessing', conn.get_database())
sql = f'''--sql
--DROP TABLE flights;
--DROP TABLE carriers;
--DROP TABLE airports;

--sql --comment included for code highlighting in IDE
 CREATE TABLE IF NOT EXISTS carriers (
    "Code" VARCHAR(7) NOT NULL,
    "Description" VARCHAR(100),
    CONSTRAINT carriers_pkey PRIMARY KEY ("Code")
);

--sql 
CREATE TABLE IF NOT EXISTS airports (
    iata VARCHAR(4) NOT NULL,
    airport VARCHAR(50) NOT NULL,
    city VARCHAR(40),
    state CHAR(2),
    country VARCHAR(40) NOT NULL,
    lat DOUBLE PRECISION NOT NULL,
    long DOUBLE PRECISION NOT NULL,
    CONSTRAINT airport_pkey PRIMARY KEY (iata)
);

CREATE TABLE IF NOT EXISTS flights (
 "Year" SMALLINT NOT NULL,  
 "Month" SMALLINT NOT NULL,  
 "DayofMonth" SMALLINT NOT NULL,  
 "DayOfWeek" SMALLINT NOT NULL,  
 "DepTime" SMALLINT,
 "CRSDepTime" SMALLINT NOT NULL,  
 "ArrTime" SMALLINT,
 "CRSArrTime" SMALLINT NOT NULL,  
 "UniqueCarrier" VARCHAR(6) NOT NULL,
 "FlightNum" SMALLINT NOT NULL,  
 "ActualElapsedTime" SMALLINT,
 "CRSElapsedTime" SMALLINT,  
 "AirTime" SMALLINT,
 "ArrDelay" SMALLINT,
 "DepDelay" SMALLINT,
 "Origin" VARCHAR(3) NOT NULL, 
 "Dest" VARCHAR(3) NOT NULL, 
 "Distance" INTEGER,
 "TaxiIn" SMALLINT,
 "TaxiOut" SMALLINT,
 "Cancelled" BOOLEAN NOT NULL,  
 "CarrierDelay" SMALLINT,
 "WeatherDelay" SMALLINT,
 "NASDelay" SMALLINT,
 "SecurityDelay" SMALLINT,
 "LateAircraftDelay" SMALLINT,
 
 -- Decided not going to need PK for this problem set. My computer is not very fast and less indexing = quicker upload
 -- I considered adding an ID column, but there aren't any problems that need a pk to solve. In a real situation, I'd always add one.
 -- PRIMARY KEY ("Year", "Month", "DayofMonth", "UniqueCarrier", "CRSDepTime", "FlightNum", "Origin", "CRSArrTime"),

 CONSTRAINT origin_airport_fk
    FOREIGN KEY ("Origin")
    REFERENCES airports(iata)
    ON DELETE CASCADE,

 CONSTRAINT dest_airport_fk 
    FOREIGN KEY ("Dest")
    REFERENCES airports(iata)
    ON DELETE CASCADE,

 CONSTRAINT uniquecarrier_carriers_fk 
    FOREIGN KEY ("UniqueCarrier")
    REFERENCES carriers("Code")
    ON DELETE CASCADE

);
'''
conn.exec(sql)
print('create successful or already created')
conn.close()

## Load Data
This section loads data into the database we just created. Since I didn't use a pk, I was careful not to add duplicates. Data was uploaded from pandas in chunks of 20,000 rows. When the process crashed, it was pretty easy to figure out where from the output and restart in the correct spot. To speed things up, I duplicated the notebook and had a couple instances running and uploading different years. 

A COPY command would have been much faster, but I had already gotten the data cleaned in pandas. It wasn't a big deal to start it and let the task run overnight. 

### Airports and Carriers

In [ ]:
## load carriers into databse

load_carriers = False ## change to true to load
if load_carriers:
    engine = create_engine(
        f'postgresql://{os.getenv("DB_USER")}:{os.getenv("DB_PASS")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{DATABASE_NAME}'
    )
    carriers_df.to_sql(
        'carriers', 
        engine, 
        if_exists='append', 
        index=False, 
        dtype={"Code":VARCHAR, "Description": VARCHAR})
else:
    print('Skipping upload')


In [ ]:
## Load Airports into database

load_airports = False ## change to true to load
if load_airports:
    engine = create_engine(
        f'postgresql://{os.getenv("DB_USER")}:{os.getenv("DB_PASS")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{DATABASE_NAME}'
    )
    airports_df.to_sql(
        'airports', 
        engine, 
        if_exists='append', 
        index=False, 
        dtype={
            "iata": VARCHAR, 
            "airport": VARCHAR,
            "city": VARCHAR,
            "state": CHAR,
            "country": VARCHAR,
            "lat": DOUBLE_PRECISION(),
            "long": DOUBLE_PRECISION(),
            })
else:
    print('Skipping upload')


### Load Flight Data

In [ ]:
## Load Flights into database

load_flights=False ## change to true to load
if load_flights:
    engine = create_engine(
        f'postgresql://{os.getenv("DB_USER")}:{os.getenv("DB_PASS")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{DATABASE_NAME}'
    )

    #speeds up bulk import
    @event.listens_for(engine, "before_cursor_execute")
    def receive_before_cursor_execute(con, cursor, statement, params, context, executemany):
        if executemany:
            cursor.fast_executemany=True

    for path_ in reversed(FLIGHT_DATA_PATHS):
        name = os.path.basename(path_)
        year = int(name.rstrip('.csv'))
        
        ## used to restart process after failure
        # if year != 2008:
        #     print(f"skipping {name}")
        #     break
        print(f'uploading {name} at {datetime.datetime.now()}')


        for chunk_num, chunk in enumerate(pd.read_csv(
            path_, chunksize=20000,encoding='ascii',dtype={
                "Year": 'int16',  
                "Month": 'int8',  
                "DayofMonth": 'int8',  
                "DayOfWeek": 'int8',  
                "DepTime": 'float64',
                "CRSDepTime": 'int16',  
                "ArrTime": 'float64',
                "CRSArrTime": 'int16',  
                "UniqueCarrier": 'string',
                "FlightNum": 'int16',  
                "ActualElapsedTime": 'float64',
                "CRSElapsedTime": 'float64',  
                "AirTime": 'float64',
                "ArrDelay": 'float64',
                "DepDelay": 'float64',
                "Origin": 'string', 
                "Dest": 'string', 
                "Distance": 'float64',
                "TaxiIn": 'float64',
                "TaxiOut": 'float64',
                "Cancelled": 'int8',  
                "CarrierDelay": 'float64',
                "WeatherDelay": 'float64',
                "NASDelay": 'float64',
                "SecurityDelay": 'float64',
                "LateAircraftDelay": 'float64',
            })):

            ## used to restart  in correct place after failure
            # if year == 2004:
            #     if chunk_num<104:
            #         continue
            
            # if year == 2007:
            #     if chunk_num!=235:
            #         print(f'skipping chunk {chunk_num}')
            #         continue
            task_start = datetime.datetime.now()
            print(f'working on chunk {chunk_num} at {task_start}')
            chunk = chunk.drop(['TailNum','CancellationCode','Diverted'], axis=1)
            chunk.to_sql(
                'flights', 
                engine, 
                if_exists='append', 
                index=False,
                method="multi",
                dtype={
                    "Year": SMALLINT,  
                    "Month": SMALLINT,  
                    "DayofMonth": SMALLINT,  
                    "DayOfWeek": SMALLINT,  
                    "DepTime": SMALLINT,
                    "CRSDepTime": SMALLINT,  
                    "ArrTime": SMALLINT,
                    "CRSArrTime": SMALLINT,  
                    "UniqueCarrier": VARCHAR,
                    "FlightNum": SMALLINT,  
                    "ActualElapsedTime": SMALLINT,
                    "CRSElapsedTime": SMALLINT,  
                    "AirTime": SMALLINT,
                    "ArrDelay": SMALLINT,
                    "DepDelay": SMALLINT,
                    "Origin": VARCHAR, 
                    "Dest": VARCHAR, 
                    "Distance": SMALLINT,
                    "TaxiIn": SMALLINT,
                    "TaxiOut": SMALLINT,
                    "Cancelled": BOOLEAN,  
                    "CarrierDelay": SMALLINT,
                    "WeatherDelay": SMALLINT,
                    "NASDelay": SMALLINT,
                    "SecurityDelay": SMALLINT,
                    "LateAircraftDelay": SMALLINT,
                    })
        complete_in = datetime.datetime.now()-task_start
        print(f'complete in {complete_in.seconds} seconds')
else:
    print('Skipping upload')

## Problem Set

#### Problem 1:	What percentage of flights were canceled each year from 1999 to 2003?

##### Code

In [43]:
conn = DBConn()
q = """
--sql
SELECT 
    count_cancelled,
    total,
    (count_cancelled/total::FLOAT)*100 perc_cancelled
FROM (
    SELECT 
        Count(*) total, 
        SUM(CASE WHEN "Cancelled" THEN 1 ELSE 0 END) count_cancelled
    FROM flights
    WHERE "Year" >= 1999 AND "Year" <= 2003
) x;
"""
ans[1] = conn.exec(q)
head[1] = ['Cancelled Flights','Total Flights', 'Percent Cancelled']
conn.close()
print(make_table(head[1], ans[1], round_cols=[2]))

connection opened ...
... connection closed.
| Cancelled Flights | Total Flights | Percent Cancelled |
| ----------------- | ------------- | ----------------- |
| 739611            | 28938610      | 2.556             |



##### Answer


| Cancelled Flights | Total Flights | Percent Cancelled |
| ----------------- | ------------- | ----------------- |
| 739611            | 28938610      | 2.556             |

#### Problem 2: On which day of the week in 2007 were you most likely to arrive on time flying from MCO to IAH

##### Code

In [44]:
conn = DBConn()
q = """
--sql
SELECT
	"DayOfWeek",
	on_time_count,
	total_flights,
	(on_time_count/total_flights::FLOAT)*100 percent_ontime
FROM
(SELECT 
	"DayOfWeek",
	SUM(CASE WHEN "CRSArrTime" <= "ArrTime" THEN 1 ELSE 0 END) on_time_count,
	COUNT(*) total_flights
FROM flights
WHERE "Year" = 2007 AND "Origin" = 'MCO' AND "Dest" = 'IAH'
GROUP BY "DayOfWeek"
) x;
"""
ans[2] = conn.exec(q)
head[2] = ['Week Day', 'On Time Flights', 'Total Flights', 'Percent On Time']
print(make_table(head[2], ans[2], round_cols=[3]))
conn.close()

connection opened ...
| Week Day | On Time Flights | Total Flights | Percent On Time  |
| -------- | --------------- | ------------- | ---------------- |
| 1        | 134             | 377           | 35.544           |
| 2        | 130             | 346           | 37.572           |
| 3        | 161             | 351           | 45.869           |
| 4        | 184             | 390           | 47.179           |
| 5        | 175             | 398           | 43.97            |
| 6        | 140             | 377           | 37.135           |
| 7        | 152             | 405           | 37.531           |

... connection closed.


##### Answer


| Week Day  | On Time Flights | Total Flights | Percent On Time  |
| --------- | --------------- | ------------- | ---------------- |
| 1 (Mon)   | 134             | 377           | 35.544           |
| 2 (Tue)   | 130             | 346           | 37.572           |
| 3 (Wed)   | 161             | 351           | 45.869           |
|**4 (Thu)**| **184**         | **390**       | **47.179**       |
| 5 (Fri)   | 175             | 398           | 43.97            |
| 6 (Sat)   | 140             | 377           | 37.135           |
| 7 (Sun)   | 152             | 405           | 37.531           |

#### Problem 3.	Which 10 flights (airline, flight number, origin city, destination city, and date) had the latest actual vs. scheduled arrival in 2004?


##### Code

In [45]:
conn = DBConn()
q = """
--sql
SELECT
	"UniqueCarrier",  -- included to make join in tableau when exported
	c."Description" AS "Airline",
	"FlightNum" AS "Flight Number",
	oa.city AS "Origin City",
	da.city AS "Dest City",
	"ArrTime" AS "Arrival Time",
	"CRSArrTime" AS "Scheduled Arrival",
	"ArrDelay" as "Arrival Delay",
	CONCAT("DayofMonth",'/',"Month",'/',"Year") AS "Date"
FROM flights
INNER JOIN carriers AS C
ON flights."UniqueCarrier" = c."Code"
INNER JOIN airports as da
ON flights."Dest" = da.iata
INNER JOIN airports as oa
ON flights."Origin" = oa.iata
WHERE "Year" = 2007 AND
	"ArrDelay" > 0 AND
	"ArrTime" IS NOT NULL AND 
	"CRSArrTime" IS NOT NULL
ORDER BY "ArrDelay" DESC
LIMIT 10
;
"""
ans[3] = conn.exec(q)
head[3] = ["Airline iata", "Airline","Flight Number", "Origin City", "Dest City","Arrival Time","Scheduled Arrival","Arrival Delay","Date"]
print(make_table(head[3],ans[3]))
conn.close()

connection opened ...
| Airline iata | Airline                 | Flight Number | Origin City     | Dest City   | Arrival Time | Scheduled Arrival | Arrival Delay | Date       |
| ------------ | ----------------------- | ------------- | --------------- | ----------- | ------------ | ----------------- | ------------- | ---------- |
| NW           | Northwest Airlines Inc. | 891           | West Palm Beach | Detroit     | 1109         | 1551              | 2598          | 3/10/2007  |
| 9E           | Pinnacle Airlines Inc.  | 4714          | Waterloo        | Minneapolis | 1627         | 805               | 1942          | 1/12/2007  |
| NW           | Northwest Airlines Inc. | 808           | Honolulu        | Minneapolis | 1244         | 556               | 1848          | 2/10/2007  |
| 9E           | Pinnacle Airlines Inc.  | 3743          | Fort Wayne      | Detroit     | 1340         | 905               | 1715          | 28/10/2007 |
| NW           | Northwest Airlines Inc. | 862  

##### Answer

| Airline                 | Flight Number | Origin City     | Dest City   | Arrival Time | Scheduled Arrival | Arrival Delay | Date       |
| ----------------------- | ------------- | --------------- | ----------- | ------------ | ----------------- | ------------- | ---------- |
| Northwest Airlines Inc. | 891           | West Palm Beach | Detroit     | 1109         | 1551              | 2598          | 3/10/2007  |
| Pinnacle Airlines Inc.  | 4714          | Waterloo        | Minneapolis | 1627         | 805               | 1942          | 1/12/2007  |
| Northwest Airlines Inc. | 808           | Honolulu        | Minneapolis | 1244         | 556               | 1848          | 2/10/2007  |
| Pinnacle Airlines Inc.  | 3743          | Fort Wayne      | Detroit     | 1340         | 905               | 1715          | 28/10/2007 |
| Northwest Airlines Inc. | 862           | Fairbanks       | Minneapolis | 2110         | 1725              | 1665          | 1/9/2007   |
| Northwest Airlines Inc. | 966           | Pittsburgh      | Detroit     | 1124         | 749               | 1655          | 28/9/2007  |
| Northwest Airlines Inc. | 189           | Madison         | Minneapolis | 1130         | 758               | 1652          | 23/12/2007 |
| Northwest Airlines Inc. | 549           | New York        | Minneapolis | 1539         | 1335              | 1564          | 16/3/2007  |
| Pinnacle Airlines Inc.  | 2925          | Austin          | Detroit     | 1424         | 1250              | 1534          | 30/9/2007  |
| Northwest Airlines Inc. | 1592          | Houston         | Memphis     | 1455         | 1323              | 1532          | 5/9/2007   |

*Not looking good for Northwest Airlines...*

#### Problem 4.	For each year from 1987 to 2008, which airline made the trip between ORD and LAX the fastest (on average)?

##### Code

In [46]:
### I'm interpreting this question as from ORD to LAX OR LAX to ORD, but the query can be modified to only include one direction by modifying the WHERE statement in CTE1 as follows:
## WHERE "Origin" = 'ORD' AND "Dest" = 'LAX'
conn = DBConn()
q = """
--sql
--First CTE returns row with average trip time per airline per year
WITH cte1 AS (
	SELECT DISTINCT
		"Year",
		"UniqueCarrier",  -- included to make join in tableau when exported
		AVG("ActualElapsedTime") OVER (
			PARTITION BY "UniqueCarrier", "Year"
			) AS avg_time
	FROM flights
	WHERE ("Origin" = 'ORD' AND "Dest" = 'LAX') OR ("Origin" = 'LAX' AND "Dest" = 'ORD')
	GROUP BY "Year", "UniqueCarrier", "ActualElapsedTime"),

--Second CTE adds rank to first
cte2 AS (
	SELECT
		"Year",
		"UniqueCarrier",
		avg_time,
		rank() OVER (PARTITION BY "Year" ORDER BY avg_time) AS annual_rank
	
	FROM cte1)

--Filter to only include top rank
SELECT
	"Year",
	"UniqueCarrier",
	carriers."Description",
	avg_time
	
FROM cte2
INNER JOIN carriers
ON cte2."UniqueCarrier" = carriers."Code"
WHERE annual_rank = 1
ORDER BY "Year"
;


"""
ans[4] = conn.exec(q)
head[4] = ["Year","iata","Airline","Trip Time"]
print(make_table(head[4],ans[4], round_cols=[3], round_to=0))
conn.close()

connection opened ...
| Year | iata | Airline                | Trip Time            |
| ---- | ---- | ---------------------- | -------------------- |
| 1987 | UA   | United Air Lines Inc.  | 250                  |
| 1988 | AA   | American Airlines Inc. | 262                  |
| 1989 | AA   | American Airlines Inc. | 266                  |
| 1990 | UA   | United Air Lines Inc.  | 263                  |
| 1991 | UA   | United Air Lines Inc.  | 265                  |
| 1992 | AA   | American Airlines Inc. | 265                  |
| 1993 | AA   | American Airlines Inc. | 268                  |
| 1994 | UA   | United Air Lines Inc.  | 258                  |
| 1995 | UA   | United Air Lines Inc.  | 261                  |
| 1996 | UA   | United Air Lines Inc.  | 271                  |
| 1997 | UA   | United Air Lines Inc.  | 276                  |
| 1998 | UA   | United Air Lines Inc.  | 267                  |
| 1999 | UA   | United Air Lines Inc.  | 283                  |
| 2000 | UA   | Un

##### Answer


| Year | Airline                | Trip Time            |
| ---- | ---------------------- | -------------------- |
| 1987 | United Air Lines Inc.  | 250                  |
| 1988 | American Airlines Inc. | 262                  |
| 1989 | American Airlines Inc. | 266                  |
| 1990 | United Air Lines Inc.  | 263                  |
| 1991 | United Air Lines Inc.  | 265                  |
| 1992 | American Airlines Inc. | 265                  |
| 1993 | American Airlines Inc. | 268                  |
| 1994 | United Air Lines Inc.  | 258                  |
| 1995 | United Air Lines Inc.  | 261                  |
| 1996 | United Air Lines Inc.  | 271                  |
| 1997 | United Air Lines Inc.  | 276                  |
| 1998 | United Air Lines Inc.  | 267                  |
| 1999 | United Air Lines Inc.  | 283                  |
| 2000 | United Air Lines Inc.  | 289                  |
| 2001 | United Air Lines Inc.  | 278                  |
| 2002 | United Air Lines Inc.  | 269                  |
| 2003 | United Air Lines Inc.  | 274                  |
| 2004 | United Air Lines Inc.  | 275                  |
| 2005 | American Airlines Inc. | 275                  |
| 2006 | United Air Lines Inc.  | 273                  |
| 2007 | American Airlines Inc. | 272                  |
| 2008 | American Airlines Inc. | 274                  |

#### Problem 5.	For the years 2002 to 2005, what is the ratio of carrier delay to elapsed travel time for each airline?

##### Code

In [47]:
conn = DBConn()
q = """
--sql
SELECT
	"UniqueCarrier", -- included to make join in tableau when exported
	carriers."Description",
	total_flights,
 	--(carrier_delay/actual_travel_time::FLOAT)*100 carrier_delay_to_actual_travel_time, --I chose to omit this, since it's like double counting the delay time
 	(carrier_delay/scheduled_travel_time::FLOAT)*100 carrier_delay_to_scheduled_travel_time
FROM(
	SELECT
		"UniqueCarrier",
		count(*) total_flights,
		sum(COALESCE("CarrierDelay",0)) carrier_delay,
		--sum("ActualElapsedTime") actual_travel_time,
		sum("CRSElapsedTime") scheduled_travel_time
	FROM flights
	WHERE "Year" >= 2002 AND
		"Year" <= 2005 AND 
		--"ActualElapsedTime" IS NOT NULL AND 
		"CRSElapsedTime" IS NOT NULL
	GROUP BY "UniqueCarrier"
 ) sub
 INNER JOIN carriers
 ON sub."UniqueCarrier" = carriers."Code"
 ORDER BY carrier_delay_to_scheduled_travel_time DESC
 ;
"""
ans[5] = conn.exec(q)
head[5] = ["iata","Airline", "Total Flights","Carrier Delay: Scheduled Flight Time"]
print(make_table(head[5],ans[5],round_cols=[3], round_to=2))
conn.close()

connection opened ...
| iata | Airline                                                                             | Total Flights | Carrier Delay: Scheduled Flight Time |
| ---- | ----------------------------------------------------------------------------------- | ------------- | ------------------------------------ |
| EV   | Atlantic Southeast Airlines                                                         | 857220        | 3.93                                 |
| OH   | Comair Inc.                                                                         | 754683        | 3.9                                  |
| OO   | Skywest Airlines Inc.                                                               | 1377703       | 3.65                                 |
| MQ   | American Eagle Airlines Inc.                                                        | 1884202       | 2.38                                 |
| DH   | Independence Air                                                     

##### Answer

| Airline                                                                             | Total Flights | Carrier Delay: Scheduled Flight Time |
| ----------------------------------------------------------------------------------- | ------------- | ------------------------------------ |
| Atlantic Southeast Airlines                                                         | 857220        | 3.93                                 |
| Comair Inc.                                                                         | 754683        | 3.9                                  |
| Skywest Airlines Inc.                                                               | 1377703       | 3.65                                 |
| American Eagle Airlines Inc.                                                        | 1884202       | 2.38                                 |
| Independence Air                                                                    | 693047        | 2.22                                 |
| Alaska Airlines Inc.                                                                | 645879        | 2.21                                 |
| Northwest Airlines Inc.                                                             | 2000609       | 1.92                                 |
| Hawaiian Airlines Inc.                                                              | 104091        | 1.87                                 |
| Expressjet Airlines Inc.                                                            | 1099556       | 1.59                                 |
| AirTran Airways Corporation                                                         | 502650        | 1.5                                  |
| Frontier Airlines Inc.                                                              | 53255         | 1.5                                  |
| America West Airlines Inc. (Merged with US Airways 9/05. Stopped reporting 10/07.)  | 779422        | 1.37                                 |
| Delta Air Lines Inc.                                                                | 2735315       | 1.21                                 |
| US Airways Inc. (Merged with America West 9/05. Reporting for both starting 10/07.) | 1769148       | 1.15                                 |
| American Airlines Inc.                                                              | 2976797       | 1.09                                 |
| ATA Airlines d/b/a ATA                                                              | 188818        | 1.01                                 |
| Southwest Airlines Co.                                                              | 3941749       | 1.0                                  |
| JetBlue Airways                                                                     | 268068        | 0.99                                 |
| United Air Lines Inc.                                                               | 2173574       | 0.99                                 |
| Continental Air Lines Inc.                                                          | 1223978       | 0.75                                 |

#### Problem 6.	What airline spent the most and least average time taxiing (in and out) at JFK in 2006?

##### Code

In [48]:
## If you just want the max and min, this query can be modified to show max and min only using a CTE, subquery, or by changing the WHERE clause. I wanted to keep all rows for vis.
conn = DBConn()
q = """
--sql
SELECT
    "UniqueCarrier",
	carriers."Description",
    COUNT(*) total_flights,
	AVG("TaxiIn" +"TaxiOut") avg_taxi
FROM flights
INNER JOIN carriers
ON flights."UniqueCarrier" = carriers."Code"
WHERE "Year" = 2006 AND
    "TaxiIn" IS NOT NULL AND 
    "TaxiOut" IS NOT NULL AND (
        "Dest" = 'JFK' OR
        "Origin" = 'JFK'
    )
        
GROUP BY "UniqueCarrier", carriers."Description"
ORDER BY avg_taxi DESC
;
"""
ans[6] = conn.exec(q)
head[6]=['iata', 'Carrier','Total Flights','Average Taxi Time (min)']
print(make_table(head[6],ans[6],round_cols=[3], round_to=0))
conn.close()

connection opened ...
| iata | Carrier                                                                             | Total Flights | Average Taxi Time (min) |
| ---- | ----------------------------------------------------------------------------------- | ------------- | ----------------------- |
| XE   | Expressjet Airlines Inc.                                                            | 698           | 43                      |
| CO   | Continental Air Lines Inc.                                                          | 1949          | 38                      |
| EV   | Atlantic Southeast Airlines                                                         | 1020          | 38                      |
| OH   | Comair Inc.                                                                         | 31594         | 37                      |
| DL   | Delta Air Lines Inc.                                                                | 25517         | 37                      |
| US   | US Airways

##### Answer

| Carrier                                                                             | Total Flights | Average Taxi Time (min) |
| ----------------------------------------------------------------------------------- | ------------- | ----------------------- |
| Expressjet Airlines Inc.                                                            | 698           | 43                      |
| Continental Air Lines Inc.                                                          | 1949          | 38                      |
| Atlantic Southeast Airlines                                                         | 1020          | 38                      |
| Comair Inc.                                                                         | 31594         | 37                      |
| Delta Air Lines Inc.                                                                | 25517         | 37                      |
| US Airways Inc. (Merged with America West 9/05. Reporting for both starting 10/07.) | 5050          | 34                      |
| Northwest Airlines Inc.                                                             | 3744          | 34                      |
| American Airlines Inc.                                                              | 24681         | 34                      |
| United Air Lines Inc.                                                               | 9797          | 31                      |
| Mesa Airlines Inc.                                                                  | 4902          | 30                      |
| JetBlue Airways                                                                     | 96604         | 30                      |
| American Eagle Airlines Inc.                                                        | 15271         | 30                      |
| ATA Airlines d/b/a ATA                                                              | 4             | 24                      |

#### Problem 7.	What were the top 10 routes (origin and destination city names and airport codes) most likely to have a weather delay of over 10 minutes in December 2005?
*Only consider routes with at least 20 flights that month.*

##### Code

In [49]:
conn = DBConn()
q = """
--sql
WITH cte1 AS (
    SELECT
        "Origin",
        "Dest",
        COUNT(*) FILTER (WHERE "WeatherDelay" > 10) AS delays,
        COUNT(*) AS total_flights
    FROM flights
    WHERE "Year" = 2005 AND "Month" = 12
    GROUP BY "Dest", "Origin"
    ),
cte2 AS (
    SELECT
        "Origin",
        "Dest",
        delays,
        total_flights,
        (delays/total_flights::FLOAT)*100 delayed_perc
    FROM cte1
    WHERE total_flights >= 20

)

SELECT
    "Origin" AS "Origin Code",
	oa.city AS "Origin City",
    "Dest" AS "Dest Code",
	da.city AS "Dest City",
    delays AS "Total Delays",
    total_flights AS "Total Flights",
    delayed_perc AS "Percent Delayed"
FROM cte2
    
INNER JOIN airports as da
ON cte2."Dest" = da.iata
INNER JOIN airports as oa
ON cte2."Origin" = oa.iata

ORDER BY delayed_perc desc
LIMIT 10
;
"""
ans[7] = conn.exec(q)
head[7] = [
    "Origin Code",
    "Origin City", 
    "Dest Code", 
    "Dest City",
    "Weather Delays",
    "Total Flights",
    "Percent Delayed"]
print(make_table(head[7],ans[7], round_cols=[6], round_to=2))
conn.close()

connection opened ...
| Origin Code | Origin City   | Dest Code | Dest City     | Weather Delays | Total Flights | Percent Delayed  |
| ----------- | ------------- | --------- | ------------- | -------------- | ------------- | ---------------- |
| BTR         | Baton Rouge   | LGA       | New York      | 8              | 26            | 30.77            |
| MEM         | Memphis       | SFO       | San Francisco | 9              | 31            | 29.03            |
| LGA         | New York      | SAV       | Savannah      | 22             | 89            | 24.72            |
| LGA         | New York      | TYS       | Knoxville     | 6              | 26            | 23.08            |
| SFO         | San Francisco | MEM       | Memphis       | 7              | 31            | 22.58            |
| LGA         | New York      | PWM       | Portland      | 19             | 92            | 20.65            |
| LGA         | New York      | BHM       | Birmingham    | 16             | 88   

##### Answer

| Origin Code | Origin City   | Dest Code | Dest City     | Weather Delays | Total Flights | Percent Delayed  |
| ----------- | ------------- | --------- | ------------- | -------------- | ------------- | ---------------- |
| BTR         | Baton Rouge   | LGA       | New York      | 8              | 26            | 30.77            |
| MEM         | Memphis       | SFO       | San Francisco | 9              | 31            | 29.03            |
| LGA         | New York      | SAV       | Savannah      | 22             | 89            | 24.72            |
| LGA         | New York      | TYS       | Knoxville     | 6              | 26            | 23.08            |
| SFO         | San Francisco | MEM       | Memphis       | 7              | 31            | 22.58            |
| LGA         | New York      | PWM       | Portland      | 19             | 92            | 20.65            |
| LGA         | New York      | BHM       | Birmingham    | 16             | 88            | 18.18            |
| ATL         | Atlanta       | MSN       | Madison       | 11             | 62            | 17.74            |
| LGA         | New York      | JAX       | Jacksonville  | 15             | 88            | 17.05            |
| CID         | Cedar Rapids  | DEN       | Denver        | 8              | 48            | 16.67            |

#### Problem 8.	Flying Southwest, what is the year-over-year change in on-time travel rate from 2000 to 2007?

##### Code

In [50]:
conn = DBConn()
q = """
--sql
--Get delayed and total flights by year for SouthWest
WITH cte1 AS (
    SELECT
        "Year",
        COUNT(*) FILTER (WHERE "ArrDelay" <= 0) AS delayed_flights,
        COUNT(*) AS total_flights
    FROM flights
    WHERE "UniqueCarrier" = 'WN' AND "Year" >= 2000 AND "Year" <= 2007
    GROUP BY "Year"
    ),
--Calculate delayed percent
cte2 AS (
    SELECT
        "Year",
        delayed_flights,
        total_flights,
        (delayed_flights/total_flights::FLOAT)*100 delayed_perc
    FROM cte1
    WHERE total_flights >= 20
)
--Window YOY
SELECT
    "Year",
    delayed_flights AS "Total Delays",
    total_flights AS "Total Flights",
    delayed_perc AS "Percent Delayed",
    (delayed_perc / lag(delayed_perc) OVER (ORDER BY "Year") - 1) * 100 AS "YOY Change"
FROM cte2

ORDER BY "Year"
;
"""
ans[8] = conn.exec(q)
head[8] = [
    "Year",
    "Delayed Flights",
    "Total Flights",
    "Percent Delayed",
    "YOY Change (%)"]
print(make_table(head[8],ans[8], round_cols=[3,4], round_to=2))
conn.close()

connection opened ...
| Year | Delayed Flights | Total Flights | Percent Delayed  | YOY Change (%)     |
| ---- | --------------- | ------------- | ---------------- | ------------------ |
| 2000 | 426194          | 911699        | 46.75            | N/A                |
| 2001 | 565652          | 957145        | 59.1             | 26.42              |
| 2002 | 575653          | 956745        | 60.17            | 1.81               |
| 2003 | 639757          | 958566        | 66.74            | 10.92              |
| 2004 | 567308          | 990404        | 57.28            | -14.18             |
| 2005 | 612598          | 1036034       | 59.13            | 3.23               |
| 2006 | 650210          | 1099321       | 59.15            | 0.03               |
| 2007 | 666644          | 1168871       | 57.03            | -3.57              |

... connection closed.


##### Answer

| Year | Delayed Flights | Total Flights | Percent Delayed  | YOY Change (%)     |
| ---- | --------------- | ------------- | ---------------- | ------------------ |
| 2000 | 426194          | 911699        | 46.75            | N/A                |
| 2001 | 565652          | 957145        | 59.1             | 26.42              |
| 2002 | 575653          | 956745        | 60.17            | 1.81               |
| 2003 | 639757          | 958566        | 66.74            | 10.92              |
| 2004 | 567308          | 990404        | 57.28            | -14.18             |
| 2005 | 612598          | 1036034       | 59.13            | 3.23               |
| 2006 | 650210          | 1099321       | 59.15            | 0.03               |
| 2007 | 666644          | 1168871       | 57.03            | -3.57              |

#### Problem 9.	What was the month-to-date on-time arrival rate for United for each date in September 2005?

##### Code

In [51]:
conn = DBConn()
q = """
--sql
-- Get delayed and total flights by year for SouthWest
WITH cte1 AS (
    SELECT
        "DayofMonth",
        COUNT(*) FILTER (WHERE "ArrDelay" <= 0) AS delayed_flights,
        COUNT(*) AS total_flights
    FROM flights 
    WHERE 
        "UniqueCarrier" = 'WN' AND 
        "Month" = 9 AND
        "Year" = 2005
    GROUP BY "DayofMonth"
    ),

-- Self join to create MTD
cte2 AS (
    SELECT
        a."DayofMonth",
        Sum(b.delayed_flights) as mtd_delayed,
        Sum(b.total_flights) as mtd_total
    FROM cte1 a
        LEFT JOIN cte1 b
            ON b."DayofMonth" <= a."DayofMonth"
    GROUP BY a."DayofMonth"
    ORDER BY a."DayofMonth"
    )
SELECT * FROM cte2
;
"""
ans[9] = conn.exec(q)
head[9] = [
    "Day",
    "Delayed Flights MTD",
    "Total Flights MTD",
    ]
print(make_table(head[9],ans[9]))
conn.close()

connection opened ...
| Day | Delayed Flights MTD | Total Flights MTD |
| --- | ------------------- | ----------------- |
| 1   | 1854                | 3020              |
| 2   | 3503                | 6042              |
| 3   | 5341                | 8489              |
| 4   | 7149                | 10647             |
| 5   | 9111                | 13375             |
| 6   | 11058               | 16322             |
| 7   | 13458               | 19266             |
| 8   | 15577               | 22212             |
| 9   | 17414               | 25160             |
| 10  | 19366               | 27642             |
| 11  | 21058               | 30333             |
| 12  | 23199               | 33277             |
| 13  | 25209               | 36227             |
| 14  | 27141               | 39173             |
| 15  | 28565               | 42120             |
| 16  | 29821               | 45068             |
| 17  | 31423               | 47327             |
| 18  | 32897               

##### Answer

| Day | Delayed Flights MTD | Total Flights MTD |
| --- | ------------------- | ----------------- |
| 1   | 1854                | 3020              |
| 2   | 3503                | 6042              |
| 3   | 5341                | 8489              |
| 4   | 7149                | 10647             |
| 5   | 9111                | 13375             |
| 6   | 11058               | 16322             |
| 7   | 13458               | 19266             |
| 8   | 15577               | 22212             |
| 9   | 17414               | 25160             |
| 10  | 19366               | 27642             |
| 11  | 21058               | 30333             |
| 12  | 23199               | 33277             |
| 13  | 25209               | 36227             |
| 14  | 27141               | 39173             |
| 15  | 28565               | 42120             |
| 16  | 29821               | 45068             |
| 17  | 31423               | 47327             |
| 18  | 32897               | 50075             |
| 19  | 34547               | 53021             |
| 20  | 36381               | 55970             |
| 21  | 38287               | 58920             |
| 22  | 39536               | 61871             |
| 23  | 40995               | 64823             |
| 24  | 42500               | 67086             |
| 25  | 44087               | 69838             |
| 26  | 46011               | 72788             |
| 27  | 48161               | 75738             |
| 28  | 50055               | 78688             |
| 29  | 51783               | 81639             |
| 30  | 53288               | 84591             |

### Export answers to .csv

In [55]:

OUTPUT_DIR = os.path.join(DATA_DIR,"output")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

ans_temp=deepcopy(ans)
head_temp=deepcopy(head)
print(os.curdir)

print('writing data to :',OUTPUT_DIR)

for i, answer in enumerate(ans_temp, 1):
    fname= "ans_"+str(i)+'.csv'
    path_ = os.path.join(OUTPUT_DIR,fname)
    print(f'writing to {path_}')
    with open(path_,"w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(head_temp[i])
        writer.writerows(ans_temp.values())

    

.
writing data to : C:\Users\Public\data\output
writing to C:\Users\Public\data\output\ans_1.csv
writing to C:\Users\Public\data\output\ans_2.csv
writing to C:\Users\Public\data\output\ans_3.csv
writing to C:\Users\Public\data\output\ans_4.csv
writing to C:\Users\Public\data\output\ans_5.csv
writing to C:\Users\Public\data\output\ans_6.csv
writing to C:\Users\Public\data\output\ans_7.csv
writing to C:\Users\Public\data\output\ans_8.csv
writing to C:\Users\Public\data\output\ans_9.csv
